In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
import random
import os
from os import listdir
from PIL import Image
from sklearn.preprocessing import label_binarize,  LabelBinarizer
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dropout, Dense
from sklearn.model_selection import train_test_split
from keras.models import model_from_json

In [ ]:
# Plotting 12 images to check dataset
#Now we will observe some of the iamges that are their in our dataset. We will plot 12 images here using the matplotlib library.
#plt.figure(figsize=(12,12))
#path = "/kaggle/input/agriculture-dataset/Agri/lemon/Diseased Lemon"
#for i in range(1,17):
#    plt.subplot(4,4,i)
#    plt.tight_layout()
#    rand_img = imread(path +'/'+ random.choice(sorted(os.listdir(path))))
#    plt.imshow(rand_img)
#    plt.xlabel(rand_img.shape[1], fontsize = 10)#width of image
#    plt.ylabel(rand_img.shape[0], fontsize = 10)#height of image

In [ ]:
import matplotlib.pyplot as plt
import random
import os
from PIL import Image

# Set the number of images to display from each subdirectory
num_images_per_subdir = 6

# Set the directory path
directory_path = "/kaggle/input/agriculture-dataset/Agri/lemon"  # Replace with your directory path

# Get the list of subdirectories
subdirectories = os.listdir(directory_path)

# Calculate the number of rows and columns for subplots
num_rows = len(subdirectories)
num_cols = num_images_per_subdir

# Create the figure and subplots
fig, axs = plt.subplots(num_rows, num_cols, figsize=(28, 28))

# Flatten the subplots array for easy indexing
axs = axs.flatten()

# Iterate over the subdirectories
for i, subdirectory in enumerate(subdirectories):
    subdir_path = os.path.join(directory_path, subdirectory)
    if not os.path.isdir(subdir_path):
        continue
    
    # Get the list of image files in the subdirectory
    image_files = os.listdir(subdir_path)
    
    # Select a random subset of images from the subdirectory
    random_images = random.sample(image_files, min(num_images_per_subdir, len(image_files)))
    
    # Iterate over the selected images
    for j, image_file in enumerate(random_images):
        ax_index = i * num_images_per_subdir + j
        ax = axs[ax_index]
        ax.axis('on')  # Remove/keep the axis labels
        
        image_path = os.path.join(subdir_path, image_file)
        image = Image.open(image_path)
        ax.imshow(image)
        ax.set_title(subdirectory)

# Remove any unused subplots
if ax_index < len(axs) - 1:
    for ax in axs[ax_index + 1:]:
        ax.remove()

# Adjust the spacing between subplots
fig.tight_layout()

# Show the plot
plt.show()


In [ ]:
#Converting Images to array 
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, (256,256))  
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from os import listdir

dir = "/kaggle/input/agriculture-dataset/Agri/lemon"
root_dir = listdir(dir)
image_list, label_list = [], []
all_labels = ['Diseased Lemon', 'Regular Lemon']
binary_labels = [0,1]
temp = -1

# Reading and converting image to numpy array
#Now we will convert all the images into numpy array.

for directory in root_dir:
  plant_image_list = listdir(f"{dir}/{directory}")
  temp += 1
  for files in plant_image_list:
    image_path = f"{dir}/{directory}/{files}"
    image_list.append(convert_image_to_array(image_path))
    label_list.append(binary_labels[temp])

In [ ]:
# Visualize the number of classes count
label_counts = pd.DataFrame(label_list).value_counts()
label_counts.head()

In [ ]:
#shape of the image.
image_list[0].shape

In [ ]:
#Checking the total number of the images which is the length of the labels list.
label_list = np.array(label_list)
label_list.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.2, random_state = 10) 

In [ ]:
#Now we will normalize the dataset of our images. As pixel values ranges from 0 to 255 so we will divide each image pixel with 255 to normalize the dataset.
x_train = np.array(x_train, dtype=np.float16) / 225.0
x_test = np.array(x_test, dtype=np.float16) / 225.0
x_train = x_train.reshape( -1, 256,256,3)
x_test = x_test.reshape( -1, 256,256,3)

In [ ]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same",input_shape=(256,256,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(16, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(8, activation="relu"))
model.add(Dense(2, activation="softmax"))
model.summary()

In [ ]:
from keras.optimizers import Adam

model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.0001),metrics=['accuracy'])

In [ ]:
epochs = 50
batch_size = 128
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

In [ ]:
#Plot the training history
plt.figure(figsize=(12, 5))
plt.plot(history.history['accuracy'], color='r')
plt.plot(history.history['val_accuracy'], color='b')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])

plt.show()

In [ ]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
!pip install Pillow

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ... previous code ...

# Convert data type to uint8
image = (x_test[10] * 255).astype(np.uint8)

# Plotting image to compare
plt.imshow(image)
plt.show()


In [ ]:
# Finding max value from predition list and comaparing original value vs predicted
print("Originally : ",all_labels[np.argmax(y_test[10])])
print("Predicted : ",all_labels[np.argmax(y_pred[10])])

In [ ]:
#import shutil

# Set the path to the directory you want to delete
#directory_path = '/kaggle/working/final'

# Use the shutil.rmtree() function to delete the directory
#shutil.rmtree(directory_path)
